# Importing Lib

In [14]:
import cv2
import eos
import numpy as np

# Open Image In Color

In [27]:
file='01_Indoor/indoor_008'

In [28]:
image = cv2.imread('image.png')
imS = cv2.resize(image, (400,400)) 
cv2.imshow('Original', imS)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Open Image In Black n White

In [29]:
image = cv2.imread('Dataset/300W/'+file+'.png')
imS = cv2.resize(image, (400,400)) 
cv2.imshow('Original', imS)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.1.0) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:3718: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


# Sharpen & Focus

In [30]:
imS = cv2.resize(image, (400,400)) 
cv2.imshow('Original', imS)
kernel_sharpening = np.array([[-1,-1,-1], 
                              [-1, 9,-1],
                              [-1,-1,-1]])
sharpened = cv2.filter2D(image, -1, kernel_sharpening)
imS = cv2.resize(sharpened, (400,400)) 
cv2.imshow('Sharp', imS)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.1.0) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:3718: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


# Noise Removal

In [31]:
imS = cv2.resize(image, (400,400)) 
cv2.imshow('Original', imS)
median = cv2.medianBlur(image,5)
imS = cv2.resize(median, (400,400)) 
cv2.imshow('Noise R', imS)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.1.0) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:3718: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


# Morphology 

In [32]:
imS = cv2.resize(image, (400,400)) 
cv2.imshow('Original', imS)
kernel = np.ones((5,5),np.uint8)
erosion = cv2.erode(image,kernel,iterations = 1)
gradient = cv2.morphologyEx(image, cv2.MORPH_GRADIENT, kernel)
imS = cv2.resize(erosion, (400,400)) 
cv2.imshow('Ero', imS)
imS = cv2.resize(gradient, (400,400)) 
cv2.imshow('Grad', imS)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.1.0) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:3718: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


# Spaitial Smoothing

In [33]:
imS = cv2.resize(image, (400,400)) 
cv2.imshow('Original', imS)
blur = cv2.GaussianBlur(image,(5,5),0)
imS = cv2.resize(blur, (400,400)) 
cv2.imshow('Spaitial', imS)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.1.0) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:3718: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


# 2D to 3D

# Select Image

In [34]:
def read_pts(filename):
    """A helper function to read the 68 ibug landmarks from a .pts file."""
    lines = open(filename).read().splitlines()
    lines = lines[3:71]

    landmarks = []
    ibug_index = 1  # count from 1 to 68 for all ibug landmarks
    for l in lines:
        coords = l.split()
        landmarks.append(eos.core.Landmark(str(ibug_index), [float(coords[0]), float(coords[1])]))
        ibug_index = ibug_index + 1

    return landmarks

In [35]:
help(eos)

Help on module eos:

NAME
    eos - Python bindings for the eos 3D Morphable Face Model fitting library.

DESCRIPTION
    For an overview of the functionality, see the documentation of the submodules. For the full documentation, see the C++ doxygen documentation.

SUBMODULES
    core
    fitting
    morphablemodel
    pca
    render

FILE
    c:\users\hp\appdata\roaming\python\python37\site-packages\eos.cp37-win_amd64.pyd




In [36]:
image = cv2.imread('image.png')
landmarks = read_pts('image.pts')

In [37]:
imS = cv2.resize(image, (400,400)) 
cv2.imshow('Original', imS)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [26]:
image_width = image.shape[1] 
image_height = image.shape[0]
model = eos.morphablemodel.load_model("eos/sfm_shape_3448.bin")
blendshapes = eos.morphablemodel.load_blendshapes("eos/expression_blendshapes_3448.bin")
morphablemodel_with_expressions = eos.morphablemodel.MorphableModel(model.get_shape_model(), blendshapes,
                                                                    color_model=eos.morphablemodel.PcaModel(),
                                                                    vertex_definitions=None,
                                                                    texture_coordinates=model.get_texture_coordinates())
landmark_mapper = eos.core.LandmarkMapper('eos/ibug_to_sfm.txt')
edge_topology = eos.morphablemodel.load_edge_topology('eos/sfm_3448_edge_topology.json')
contour_landmarks = eos.fitting.ContourLandmarks.load('eos/ibug_to_sfm.txt')
model_contour = eos.fitting.ModelContour.load('eos/sfm_model_contours.json')

(mesh, pose, shape_coeffs, blendshape_coeffs) = eos.fitting.fit_shape_and_pose(morphablemodel_with_expressions,
                                                                               landmarks, landmark_mapper, image_width,
                                                                               image_height, edge_topology,
                                                                               contour_landmarks, model_contour)
model_3d = eos.core.Mesh()
model_3d = eos.morphablemodel.sample_to_mesh(model.get_shape_model().draw_sample(shape_coeffs), model.get_color_model().get_mean(), model.get_shape_model().get_triangle_list(), model.get_color_model().get_triangle_list(), model.get_texture_coordinates());
eos.core.write_textured_obj(mesh=model_3d,filename='3dmesh.obj')
isomap = eos.render.extract_texture(mesh, pose, image)
rows = isomap.shape[0]
cols=isomap.shape[1]
#kernel = np.ones((5,5),np.uint8)
#gradient = cv2.morphologyEx(isomap, cv2.MORPH_GRADIENT, kernel)
M = cv2.getRotationMatrix2D((cols/2,rows/2),-90,1)
isomap = cv2.warpAffine(isomap,M,(cols,rows))
cv2.imwrite("3dmesh.isomap.png", isomap)

True